# Tracking Bird Migration Routes
### Milestone 2
#### Cloe Kwiatkowski

# !['Bird Migration'](navbynature.jpeg)

### Objective: 
    Using the 'bird_data.sql' and 'city_weather.sql' tables, join them using a SQL script and import the new csv file into a Jupyter Notebook for further analysis. Determine the optimal bird-watching trip location(s) for each of the types of travelers listed below.

**Background**

*Wings 4 Life* is a bird refuge organization that helps bird watchers plan bird-watching trips along migration routes on the southwestern coast of Europe and northwestern coast of Africa. 

It provides three different travel packages for tourists to choose from based on the locations they wish to visit and the types of birds they would like to see. You will build a map of the best bird-watching sites for the three different traveler profiles listed below:


**The French Traveler**
* Only visits locations within or very near France
* Prefers high-speed bird sightings
* Hates cold weather

**The Retirement Getaway**
* Prefers to travel in the spring months
* Prefers birds that fly at low altitudes
* Willing to visit any location with mild weather (60s–70s)

**The Loner**
* Prefers the early morning 
* Likes cool weather
* Looking for higher-altitude bird flights 

In [4]:
# imports
import pandas as pd
import numpy as np
import plotly.express as px

#### Importing CSV file of table created in MySQL

In [5]:
# creating dataframe from birds csv file
birds = pd.read_csv('bird_tracking.csv')

In [6]:
birds.isnull().sum()  # returns number of missing cells for each column
birds.dropna(inplace=True)  # will drop any records with missing data
birds.head()

,id,altitude,date_time,device_info_serial,direction,latitude,longitude,speed_2d,bird_name,nearest_city,country,avg_temp
0,1,71,8/14/2013 19:18,851,-150.469753,49.4199,2.12073,0.150000,Eric,PARIS,FRANCE,63.5
1,2,73,8/14/2013 20:47,851,32.769360,49.4204,2.12086,0.310161,Eric,PARIS,FRANCE,63.5
2,3,54,8/14/2013 21:47,851,-46.344478,49.4204,2.12084,2.904772,Eric,PARIS,FRANCE,63.5
3,4,59,8/14/2013 22:32,851,-57.682428,49.4203,2.12086,2.640076,Eric,PARIS,FRANCE,63.5
4,5,57,8/14/2013 23:33,851,-110.181448,49.4210,2.12136,1.897630,Eric,PARIS,FRANCE,63.5


In [7]:
# checking for duplicates; if there are duplicates use df.drop_duplicates()
birds.duplicated().sum()

0

In [8]:
birds.dtypes  # what are our data types in each column
birds.describe()  # give more info on data set

,id,altitude,device_info_serial,direction,latitude,longitude,speed_2d,avg_temp
count,9930.000000,9930.000000,9930.000000,9930.000000,9930.000000,9930.000000,9930.000000,9930.000000
mean,4997.300403,51.344713,849.398590,-6.298123,30.428889,-8.829069,2.527014,64.125237
std,2889.078150,131.862844,12.787991,102.741537,14.841404,8.511616,3.571868,16.380509
min,1.000000,-668.000000,833.000000,-179.985721,12.358400,-17.624600,0.000000,-99.000000
25%,2497.250000,3.000000,833.000000,-91.342204,15.549475,-16.758075,0.402150,55.700000
50%,4990.500000,14.000000,851.000000,-12.185866,30.428600,-9.652620,1.192434,65.500000
75%,7491.750000,87.000000,864.000000,80.327169,50.034450,2.619160,2.994178,71.700000
max,9999.000000,5448.000000,864.000000,180.000000,51.514300,4.853160,63.488066,92.800000


In [9]:
# histogram of avg_temp
fig = px.histogram(birds, x='avg_temp', range_x=[0,100], title= 'Average Temperature Histogram')
fig.show()
# outlier around the zero mark; minimum with an average temp of -99.000000

In [10]:
# altitude histogram
fig = px.histogram(birds, x='altitude', range_x=[-200,200], title='Altitude Histogram')
fig.show()
# >=15% of the data is less than 0 but distribution is valid; no outliers in altitude column

In [11]:
# speed_2x histogram
fig = px.histogram(birds, x='speed_2d', range_x=[0,15], title='Speed Histogram')
fig.show()
# reasonable distribution; no outliers

In [12]:
# dropping outliers in the temperature column
birds = birds[birds['avg_temp']>40]
birds.head()

,id,altitude,date_time,device_info_serial,direction,latitude,longitude,speed_2d,bird_name,nearest_city,country,avg_temp
0,1,71,8/14/2013 19:18,851,-150.469753,49.4199,2.12073,0.150000,Eric,PARIS,FRANCE,63.5
1,2,73,8/14/2013 20:47,851,32.769360,49.4204,2.12086,0.310161,Eric,PARIS,FRANCE,63.5
2,3,54,8/14/2013 21:47,851,-46.344478,49.4204,2.12084,2.904772,Eric,PARIS,FRANCE,63.5
3,4,59,8/14/2013 22:32,851,-57.682428,49.4203,2.12086,2.640076,Eric,PARIS,FRANCE,63.5
4,5,57,8/14/2013 23:33,851,-110.181448,49.4210,2.12136,1.897630,Eric,PARIS,FRANCE,63.5


In [13]:
# Updated histogram of avg_temp after removing outlier
fig = px.histogram(birds, x='avg_temp', title='Updated Average Temperature Histogram After Removing Outliers')
fig.show()

In [14]:
# descriptive stats for bird_name column
birds[['bird_name']].value_counts()

bird_name
Nico         3355
Sanne        3313
Eric         3203
dtype: int64

In [15]:
# descriptive stats for nearest_city column
birds[['nearest_city']].value_counts()  

nearest_city
DAKAR           2917
RABAT           2003
PARIS           1554
BRUSSELS        1461
NOUAKCHOTT      1274
BISSAU           247
LISBON           182
BANJUL           108
BILBAO            66
BORDEAUX          59
dtype: int64

In [16]:
# calculating average temperature for each country in the data set
tracking_birds = birds[['country', 'avg_temp']]  # creating new dataframe with country and avg_temp columns
tracking_birds.head()

,country,avg_temp
0,FRANCE,63.5
1,FRANCE,63.5
2,FRANCE,63.5
3,FRANCE,63.5
4,FRANCE,63.5


In [17]:
# calculating average temperature per city using groupby()
tracking_birds.groupby('country').mean().sort_values(['avg_temp'], ascending=True)

,avg_temp
country,
MOROCCO,56.151972
BELGIUM,56.395277
SPAIN,58.148485
FRANCE,59.940112
PORTUGAL,65.350000
SENEGAL,72.529208
MAURITANIA,74.682653
GAMBIA,76.791667
GUINEA-BISSAU,82.254656


In [18]:
# dropping unnecessary columns - id, device_info_serial, direction, and nearest_city columns
birds.drop(['id', 'device_info_serial', 'direction', 'nearest_city'], axis=1, inplace=True)

# renaming speed_2x to speed
birds['speed'] = birds['speed_2d']

# viewing table to confirm changes
birds.head()

,altitude,date_time,latitude,longitude,speed_2d,bird_name,country,avg_temp,speed
0,71,8/14/2013 19:18,49.4199,2.12073,0.150000,Eric,FRANCE,63.5,0.150000
1,73,8/14/2013 20:47,49.4204,2.12086,0.310161,Eric,FRANCE,63.5,0.310161
2,54,8/14/2013 21:47,49.4204,2.12084,2.904772,Eric,FRANCE,63.5,2.904772
3,59,8/14/2013 22:32,49.4203,2.12086,2.640076,Eric,FRANCE,63.5,2.640076
4,57,8/14/2013 23:33,49.4210,2.12136,1.897630,Eric,FRANCE,63.5,1.897630


#### Subsequent Visualizations In Tableau:
https://public.tableau.com/app/profile/cloe.kwiatkowski/viz/BirdTrackingDashboard/Dashboard1#1